In [1]:
import spacy
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
nlp1 = spacy.load("pl_document_ner/output/model-best")

c:\users\jaceb\studia\magisterka - sem 2\iui\venv\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'pl_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [9]:
df =pd.read_csv("DATASET_NO_UNUSUAL_AND_INTERPOLSKA.csv")
df

,Unnamed: 0.1,Unnamed: 0,categories,text,text_full,main_category,label_mid,label_high
0,0,0,114 Rzeczowe,['czy wspólnota mieszkaniowa może podjąć uchwa...,czy wspólnota mieszkaniowa może podjąć uchwałę...,rzeczowe,rzeczowe,prawo cywilne
1,1,1,114 Rzeczowe,['czy wspólnota mieszkaniowa może odzyskać pom...,czy wspólnota mieszkaniowa może odzyskać pomie...,rzeczowe,rzeczowe,prawo cywilne
2,2,3,15 Prawo ochrony danych osobowych,['czy uprawnienie mieszkańców wspólnoty mieszk...,czy uprawnienie mieszkańców wspólnoty mieszkan...,prawo ochrony danych osobowych,prawo administracyjne materialne,prawo administracyjne
3,3,4,"20 Prawo budowlane, 114 Rzeczowe",['czy budowa windy dla osób niepełnosprawnych ...,czy budowa windy dla osób niepełnosprawnych mo...,prawo budowlane,prawo administracyjne materialne,prawo administracyjne
4,4,5,114 Rzeczowe,['jakie czynności powinna podjąć wspólnota mie...,jakie czynności powinna podjąć wspólnota miesz...,rzeczowe,rzeczowe,prawo cywilne
...,...,...,...,...,...,...,...,...
2669,2716,4030,77 Ustawa o prawach pacjenta i Rzeczniku Praw ...,['czy dokumentacja medyczna powinna zostać wyd...,czy dokumentacja medyczna powinna zostać wydan...,ustawa o prawach pacjenta i rzeczniku praw pac...,ustawa o prawach pacjenta i rzeczniku praw pac...,prawo medyczne
2670,2717,4031,"59 Ustawa Prawo farmaceutyczne, 197 Inne - pra...","['czy apteka może sprzedawać wodę mineralną?',...",czy apteka może sprzedawać wodę mineralną czy ...,ustawa prawo farmaceutyczne,ustawa prawo farmaceutyczne,prawo farmaceutyczne
2671,2718,4032,"15 Prawo ochrony danych osobowych, 69 Prawo ka...",['co w przedmiotowej sprawie może zrobić klien...,co w przedmiotowej sprawie może zrobić klientka,prawo ochrony danych osobowych,prawo administracyjne materialne,prawo administracyjne
2672,2719,4033,79 Ustawa o zawodach lekarza i lekarza dentyst...,['czy lekarz stażysta może przeprowadzać kwali...,czy lekarz stażysta może przeprowadzać kwalifi...,ustawa o zawodach lekarza i lekarza dentysty,ustawa o zawodach lekarza i lekarza dentysty,prawo medyczne


In [10]:
def classifyText(text):
    doc = nlp1(text)
    legalAffairs = OrderedDict()
    for token in doc:
        if token.ent_type_ != '':
            if token.ent_type_ not in legalAffairs:
                legalAffairs[token.ent_type_] = 0
            legalAffairs[token.ent_type_] += 1
    legalAffairs = OrderedDict(sorted(legalAffairs.items(),key = lambda key: key[1], reverse=True))
    if len(legalAffairs) == 0:
        return 'UNKNOWN'
    # print(legalAffairs)
    return str(next(iter(legalAffairs)))


In [11]:
correctClassifications = 0
zeroLabelsClassifications = 0
labelTags = {'administrative_law':'prawo administracyjne','civil_law':'prawo cywilne','prawo_medyczne':'medical_law','tax_law':'prawo podatkowe','criminal_law':'prawo karne','pharmaceutical_law':'prawo farmaceutyczne', 'labor_law':'prawo pracy','international_law':'prawo miädzynarodowe','constitutional_law':'prawo konstytucyjne'}
for i in range(len(df)):
    label= df['label_high'][i]
    classified = 'UNKNOWN'
    labelTag = classifyText(df['text'][i])
    if labelTag in labelTags.keys():
        classified = labelTags[labelTag]
    print('{}/{} label: {}, classified: {} | CORRECT?: {}'.format(i,len(df),label,classified ,label==classified))
    if label == classified:
        correctClassifications += 1
    if classified == 'UNKNOWN':
        zeroLabelsClassifications +=1
print('Overall accuracy: {}%'.format(correctClassifications/len(df)*100))
print('Textes with zero labels in classification: {}, percent of all: {}%'.format(zeroLabelsClassifications,zeroLabelsClassifications/len(df)*100))

0/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
1/2674 label: prawo cywilne, classified: prawo administracyjne | CORRECT?: False
2/2674 label: prawo administracyjne, classified: prawo cywilne | CORRECT?: False
3/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
4/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
5/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
6/2674 label: prawo administracyjne, classified: UNKNOWN | CORRECT?: False
7/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
8/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
9/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
10/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
11/2674 label: prawo cywilne, classified: prawo pracy | CORRECT?: False
12/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
13/2674 label: prawo admi

110/2674 label: prawo administracyjne, classified: UNKNOWN | CORRECT?: False
111/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
112/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
113/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
114/2674 label: prawo karne, classified: prawo cywilne | CORRECT?: False
115/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
116/2674 label: prawo pracy, classified: prawo pracy | CORRECT?: True
117/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
118/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
119/2674 label: prawo karne, classified: prawo karne | CORRECT?: True
120/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
121/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
122/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
123/2674 label: prawo administracyjne, classified:

232/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
233/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
234/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
235/2674 label: prawo farmaceutyczne, classified: UNKNOWN | CORRECT?: False
236/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
237/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
238/2674 label: prawo pracy, classified: prawo pracy | CORRECT?: True
239/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
240/2674 label: prawo pracy, classified: UNKNOWN | CORRECT?: False
241/2674 label: prawo medyczne, classified: prawo administracyjne | CORRECT?: False
242/2674 label: prawo karne, classified: prawo karne | CORRECT?: True
243/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
244/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
245/2674 label: prawo medyczne, classified: UNKNOWN | 

396/2674 label: prawo podatkowe, classified: prawo cywilne | CORRECT?: False
397/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
398/2674 label: prawo cywilne, classified: UNKNOWN | CORRECT?: False
399/2674 label: prawo pracy, classified: prawo administracyjne | CORRECT?: False
400/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
401/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
402/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
403/2674 label: prawo cywilne, classified: UNKNOWN | CORRECT?: False
404/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
405/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
406/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
407/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
408/2674 label: prawo pracy, classified: UNKNOWN | CORRECT?: False
409/2674 label: prawo administracyjne, classifi

520/2674 label: prawo medyczne, classified: prawo administracyjne | CORRECT?: False
521/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
522/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
523/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
524/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
525/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
526/2674 label: prawo administracyjne, classified: prawo pracy | CORRECT?: False
527/2674 label: prawo administracyjne, classified: prawo cywilne | CORRECT?: False
528/2674 label: prawo cywilne, classified: UNKNOWN | CORRECT?: False
529/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
530/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
531/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
532/2674 label: prawo farmaceutyczne, classified: prawo cywilne | CORRECT?: Fals

...

2118/2674 label: prawo karne, classified: prawo cywilne | CORRECT?: False
2119/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2120/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2121/2674 label: prawo pracy, classified: prawo pracy | CORRECT?: True
2122/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2123/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
2124/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2125/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2126/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2127/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2128/2674 label: prawo farmaceutyczne, classified: prawo farmaceutyczne | CORRECT?: True
2129/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2130/2674 label: prawo administracyjne, classified: prawo cywilne | CORRE

2232/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2233/2674 label: prawo pracy, classified: prawo cywilne | CORRECT?: False
2234/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2235/2674 label: prawo administracyjne, classified: UNKNOWN | CORRECT?: False
2236/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2237/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2238/2674 label: prawo administracyjne, classified: prawo cywilne | CORRECT?: False
2239/2674 label: prawo karne, classified: prawo karne | CORRECT?: True
2240/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2241/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
2242/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
2243/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2244/2674 label: prawo medyczne, classified: UNKNOWN | CORR

2398/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2399/2674 label: prawo podatkowe, classified: prawo cywilne | CORRECT?: False
2400/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2401/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2402/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2403/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2404/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2405/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
2406/2674 label: prawo administracyjne, classified: prawo cywilne | CORRECT?: False
2407/2674 label: prawo farmaceutyczne, classified: UNKNOWN | CORRECT?: False
2408/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2409/2674 label: prawo pracy, classified: prawo pracy | CORRECT?: True
2410/2674 label: prawo miädzynarodowe, classified: prawo cywilne | CORRECT?: False
2411/

2515/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2516/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2517/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2518/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2519/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2520/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2521/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2522/2674 label: prawo farmaceutyczne, classified: prawo administracyjne | CORRECT?: False
2523/2674 label: prawo pracy, classified: UNKNOWN | CORRECT?: False
2524/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2525/2674 label: prawo medyczne, classified: prawo administracyjne | CORRECT?: False
2526/2674 label: prawo medyczne, classified: UNKNOWN | CORRECT?: False
2527/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2528/2674 label: prawo k

2624/2674 label: prawo pracy, classified: prawo pracy | CORRECT?: True
2625/2674 label: prawo pracy, classified: UNKNOWN | CORRECT?: False
2626/2674 label: prawo karne, classified: UNKNOWN | CORRECT?: False
2627/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2628/2674 label: prawo administracyjne, classified: UNKNOWN | CORRECT?: False
2629/2674 label: prawo pracy, classified: prawo pracy | CORRECT?: True
2630/2674 label: prawo medyczne, classified: prawo administracyjne | CORRECT?: False
2631/2674 label: prawo medyczne, classified: prawo cywilne | CORRECT?: False
2632/2674 label: prawo cywilne, classified: prawo cywilne | CORRECT?: True
2633/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
2634/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
2635/2674 label: prawo administracyjne, classified: prawo administracyjne | CORRECT?: True
2636/2674 label: prawo pracy, classified: prawo pracy | COR

In [177]:
d= classifyText("czy wspólnota mieszkaniowa może podjąć uchwałę dotyczącą zakupu wodomierzy odczytywanych radiowo do lokali pokrywając ich koszty czy wspólnota mieszkaniowa może odzyskać pomieszczenie gospodarcze jakie kroki wspólnota mieszkaniowa powinna podjąć aby odzyskać przedmiotowe pomieszczenie czy uprawnienie mieszkańców wspólnoty mieszkaniowej jest zgodne z przepisami rodo czy budowa windy dla osób niepełnosprawnych może zostać zakwalifikowana jako remont i zostać sfinansowana z funduszu remontowego dużej wspólnoty mieszkaniowej jakie czynności powinna podjąć wspólnota mieszkaniowa założenie funduszu celowego w celu wybudowania windy dla osób niepełnosprawnych czy wspólnota mieszkaniowa może uchwałą wspólników przeksięgować środki z funduszu remontowego na fundusz celowy czy nieruchomość nabywa wspólnota mieszkaniowa czy też jej poszczególni członkowie kto będzie właścicielem nabytej nieruchomości kto będzie zobowiązany do zapłaty podatku od nieruchomości czy na zakup nieruchomości wspólnota może przeznaczyć środki zgromadzone na zaliczce remontowo inwestycyjnej czy do nabycia nieruchomości wystarczy podjęcie uchwały większością głosów jaką treść powinny zawierać przedmiotowe tablice informacyjne jakie koszty musi ponieść wspólnota mieszkaniowa w związku ze złożeniem wniosku o stwierdzenie nabycia spadku w którego skład wchodzi nieruchomość pozostająca we wspólnocie czy z uwagi na uchybienie formalne podejmowanie uchwały poprzez indywidualne zbieranie głosów od członków wspólnoty mieszkaniowej które zostało dokonane przez inny podmiot niż zarząd lub zarządcę nieruchomości wspólnej istnieje przesłanka do stwierdzenia jej nieważności czy pomimo tego iż uchwała została podjęta większością głosów to nadal istnieje obowiązek dalszego zbierania głosów czy w treści uchwały należy zamieszczać rubrykę informującą o wstrzymaniu się od głosu w jaki sposób odpowiedzieć na zarzut iż sposób zbierania głosów mógł mieć istotny wpływ na treść uchwały czy zarząd wspólnoty mieszkaniowej ma możliwość wyjścia z inicjatywą anulowania uchwały a następnie zbierania głosów pod identyczną uchwałą z zachowaniem wymogów formalnych czy wspólnota może rozłożyć właścicielom lokali płatność na raty czy z każdym właścicielem lokalu należy zawrzeć odrębną umowę czy wspólnota")
print(d)

OrderedDict([('criminal_law', 2), ('tax_law', 2), ('pharmaceutical_law', 1), ('medical_law', 1)])
criminal_law


In [10]:
doc = nlp1("czy wspólnota mieszkaniowa może podjąć uchwałę dotyczącą zakupu wodomierzy odczytywanych radiowo do lokali pokrywając ich koszty czy wspólnota mieszkaniowa może odzyskać pomieszczenie gospodarcze jakie kroki wspólnota mieszkaniowa powinna podjąć aby odzyskać przedmiotowe pomieszczenie czy uprawnienie mieszkańców wspólnoty mieszkaniowej jest zgodne z przepisami rodo czy budowa windy dla osób niepełnosprawnych może zostać zakwalifikowana jako remont i zostać sfinansowana z funduszu remontowego dużej wspólnoty mieszkaniowej jakie czynności powinna podjąć wspólnota mieszkaniowa założenie funduszu celowego w celu wybudowania windy dla osób niepełnosprawnych czy wspólnota mieszkaniowa może uchwałą wspólników przeksięgować środki z funduszu remontowego na fundusz celowy czy nieruchomość nabywa wspólnota mieszkaniowa czy też jej poszczególni członkowie kto będzie właścicielem nabytej nieruchomości kto będzie zobowiązany do zapłaty podatku od nieruchomości czy na zakup nieruchomości wspólnota może przeznaczyć środki zgromadzone na zaliczce remontowo inwestycyjnej czy do nabycia nieruchomości wystarczy podjęcie uchwały większością głosów jaką treść powinny zawierać przedmiotowe tablice informacyjne jakie koszty musi ponieść wspólnota mieszkaniowa w związku ze złożeniem wniosku o stwierdzenie nabycia spadku w którego skład wchodzi nieruchomość pozostająca we wspólnocie czy z uwagi na uchybienie formalne podejmowanie uchwały poprzez indywidualne zbieranie głosów od członków wspólnoty mieszkaniowej które zostało dokonane przez inny podmiot niż zarząd lub zarządcę nieruchomości wspólnej istnieje przesłanka do stwierdzenia jej nieważności czy pomimo tego iż uchwała została podjęta większością głosów to nadal istnieje obowiązek dalszego zbierania głosów czy w treści uchwały należy zamieszczać rubrykę informującą o wstrzymaniu się od głosu w jaki sposób odpowiedzieć na zarzut iż sposób zbierania głosów mógł mieć istotny wpływ na treść uchwały czy zarząd wspólnoty mieszkaniowej ma możliwość wyjścia z inicjatywą anulowania uchwały a następnie zbierania głosów pod identyczną uchwałą z zachowaniem wymogów formalnych czy wspólnota może rozłożyć właścicielom lokali płatność na raty czy z każdym właścicielem lokalu należy zawrzeć odrębną umowę czy wspólnota")

In [11]:
spacy.displacy.render(doc, style="ent", jupyter=True)